IMPORTATIONS

In [1]:
pip install -U imbalanced-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#IMPORTATIONS GENERALES
import csv
import numpy as np
import pandas as pd
import os
import nltk
#import matplotlib.pyPhrase as plt
import pickle
#SKLEARN 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix

   #LES 4 MODELES
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
#AUTRE
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

import seaborn as sns

# NLTK
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\incen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\incen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


FONCTIONS ET DICO

In [6]:

def preprocess_comma(file,newfile):
    data_frame = pd.read_csv(file)

    # Retirer les virgules sauf la première de chaque ligne
    data_frame.iloc[:, 1:] =  data_frame.iloc[:, 1:].replace(to_replace=',', value='', regex=True)
# Enregistrer dans un nouveau fichier CSV avec un autre nom
    nom_fichier_sortie = newfile
    data_frame.to_csv(nom_fichier_sortie, index=False)
    return data_frame

    

models = {
    'SVM': SVC(class_weight='balanced'),
    'Random Forest': RandomForestClassifier(class_weight='balanced'),
    'Multinomial Naive Bayes': MultinomialNB(),
    'Perceptron': Perceptron(class_weight='balanced')
}

def printresult(model_name,accuracy,classification_report_result):
    print(f'\nResults for {model_name}:')
    print(f'Accuracy: {accuracy}')
    print('Classification Report:')
    print(classification_report_result)
    print('-' * 50)


def evaluation(predictions,y_test):
    accuracy = accuracy_score(y_test, predictions)
    classification_report_result = classification_report(y_test, predictions)
    return accuracy, classification_report_result

def lemmatiser(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

PREMIERE MISE EN FORME DES DONNEES

In [15]:


#dataiemocap= dataiemocap.sample(n=1000, random_state=7)

data_path = 'IEMOCAP_features.pkl'

# users should use this instructor to load pkl dataset. 
videoIDs, videoSpeakers, videoLabels, videoText,\
    videoAudio, videoVisual, videoSentence, trainVid,\
        testVid = pickle.load(open(data_path, 'rb'), encoding='latin1')
#IEMOCAP 
#ON MET DANS UN CSV

data_list = []
for video_id, phrases in videoSentence.items():
    emotions = videoLabels.get(video_id, [])  
    for emotion, phrase in zip(emotions, phrases):
        data_list.append({'Emotion': emotion, 'Phrase': phrase})
df = pd.DataFrame(data_list)
df.to_csv('output.csv', index=False)

dataiemocap = preprocess_comma("output.csv","dataiemocap.csv")

print(len(dataiemocap))



7433


In [12]:
leng = dataiemocap['Phrase'][0]
leng = leng.split()
print(len(leng))
print(leng)


5
["You're", 'not', 'sorry', 'you', 'came?']


In [16]:




X = dataiemocap['Phrase']
y = dataiemocap['Emotion']
print(len(X))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)
print(len(X_train))
for model_name, model in models.items():
    print(f"Training {model_name}...")
    pipeline1 = make_pipeline(CountVectorizer(), model)
    pipeline2 = make_pipeline(TfidfVectorizer(), model)
# Entraîner le modèle
    pipeline1.fit(X_train, y_train)
    pipeline2.fit(X_train, y_train)
# Prédictions sur l'ensemble de test
    predictions1 = pipeline1.predict(X_test)
    predictions2 = pipeline2.predict(X_test)
    
# Évaluation des performances du modèle avec COUNTVECTORIZER 
    accuracy1 , classification_report_result1 = evaluation(predictions1,y_test)
    accuracy2 , classification_report_result2 = evaluation(predictions2,y_test)
    # Affichage
    print("-----------------COUNTVECTORIZER------------------")
    printresult(model_name,accuracy1,classification_report_result1)
    print("---------------------TFIDF------------------------")
    printresult(model_name,accuracy2,classification_report_result2)
    


7433
5203
Training SVM...
-----------------COUNTVECTORIZER------------------

Results for SVM:
Accuracy: 0.2789237668161435
Classification Report:
              precision    recall  f1-score   support

           0       0.39      0.11      0.17       178
           1       0.50      0.16      0.24       315
           2       0.23      0.88      0.37       509
           3       0.68      0.10      0.17       342
           4       0.43      0.04      0.07       337
           5       0.67      0.10      0.18       549

    accuracy                           0.28      2230
   macro avg       0.48      0.23      0.20      2230
weighted avg       0.49      0.28      0.21      2230

--------------------------------------------------
---------------------TFIDF------------------------

Results for SVM:
Accuracy: 0.5417040358744395
Classification Report:
              precision    recall  f1-score   support

           0       0.39      0.40      0.39       178
           1       0.54      

ON VA SURECHANTILLONNER LES CLASSES SOUS REPRESENTEES

In [16]:
X = dataiemocap['Phrase']
y = dataiemocap['Emotion']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# Vectoriseurs à tester
vectorizers = {
    'CountVectorizer': CountVectorizer(),
    'TfidfVectorizer': TfidfVectorizer()
}

for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())        

Training SVM with CountVectorizer...
-----------------SVM with CountVectorizer------------------

Results for SVM:
Accuracy: 0.44484304932735425
Classification Report:
              precision    recall  f1-score   support

           0       0.22      0.37      0.28       178
           1       0.51      0.49      0.50       315
           2       0.51      0.40      0.45       509
           3       0.50      0.47      0.49       342
           4       0.46      0.35      0.40       337
           5       0.44      0.52      0.48       549

    accuracy                           0.44      2230
   macro avg       0.44      0.43      0.43      2230
weighted avg       0.46      0.44      0.45      2230

--------------------------------------------------
Training Random Forest with CountVectorizer...
-----------------Random Forest with CountVectorizer------------------

Results for Random Forest:
Accuracy: 0.48654708520179374
Classification Report:
              precision    recall  f1-sc

LES N-GRAM POUR PLUS DE CONTEXTE

UNI ET BI GRAM

In [17]:
X = dataiemocap['Phrase']
y = dataiemocap['Emotion']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# ------------------------Vectoriseurs à tester + ON RAJOUTE LES NGRAM C'EST LA SEULE DIFFERENCE DANS CETTE CELLULE-------------
vectorizers = {
    'CountVectorizer': CountVectorizer(ngram_range=(1, 2)),
    'TfidfVectorizer': TfidfVectorizer(ngram_range=(1, 2))
}
#~-----------------------------------------------------------------------------------------------------------------------------
for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())        

Training SVM with CountVectorizer...
-----------------SVM with CountVectorizer------------------

Results for SVM:
Accuracy: 0.4385650224215247
Classification Report:
              precision    recall  f1-score   support

           0       0.17      0.38      0.24       178
           1       0.54      0.46      0.50       315
           2       0.47      0.36      0.41       509
           3       0.57      0.46      0.51       342
           4       0.55      0.31      0.40       337
           5       0.44      0.58      0.50       549

    accuracy                           0.44      2230
   macro avg       0.46      0.43      0.43      2230
weighted avg       0.48      0.44      0.45      2230

--------------------------------------------------
Training Random Forest with CountVectorizer...
-----------------Random Forest with CountVectorizer------------------

Results for Random Forest:
Accuracy: 0.4802690582959641
Classification Report:
              precision    recall  f1-scor

UNI BI ET TRI

In [18]:
X = dataiemocap['Phrase']
y = dataiemocap['Emotion']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# ------------------------Vectoriseurs à tester + ON RAJOUTE LES NGRAM C'EST LA SEULE DIFFERENCE DANS CETTE CELLULE-------------
vectorizers = {
    'CountVectorizer': CountVectorizer(ngram_range=(1, 3)),
    'TfidfVectorizer': TfidfVectorizer(ngram_range=(1, 3))
}
#~-----------------------------------------------------------------------------------------------------------------------------
for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())        

Training SVM with CountVectorizer...
-----------------SVM with CountVectorizer------------------

Results for SVM:
Accuracy: 0.420627802690583
Classification Report:
              precision    recall  f1-score   support

           0       0.16      0.40      0.23       178
           1       0.53      0.45      0.49       315
           2       0.46      0.35      0.40       509
           3       0.54      0.45      0.49       342
           4       0.56      0.28      0.38       337
           5       0.44      0.54      0.48       549

    accuracy                           0.42      2230
   macro avg       0.45      0.41      0.41      2230
weighted avg       0.47      0.42      0.43      2230

--------------------------------------------------
Training Random Forest with CountVectorizer...
-----------------Random Forest with CountVectorizer------------------

Results for Random Forest:
Accuracy: 0.4896860986547085
Classification Report:
              precision    recall  f1-score

ENLEVER STOP WORDS ?

In [22]:
# ON VA UTILISER LES STOPSWORDS DE NLTK
stop_words = set(stopwords.words('english'))

X = dataiemocap['Phrase']
y = dataiemocap['Emotion']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

#-------------------- Vectoriseurs à tester + ON PEUX METTRE LES STOPWORDS EN ARGUMENT DU VECTORISEUR ----------------------------------
vectorizers = {
    'CountVectorizer': CountVectorizer(stop_words=stop_words),
    'TfidfVectorizer': TfidfVectorizer(stop_words=stop_words)
}
#--------------------------------------------------------------
for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())   

Training SVM with CountVectorizer...
-----------------SVM with CountVectorizer------------------

Results for SVM:
Accuracy: 0.4031390134529148
Classification Report:
              precision    recall  f1-score   support

           0       0.17      0.37      0.23       178
           1       0.46      0.41      0.43       315
           2       0.46      0.36      0.41       509
           3       0.53      0.46      0.49       342
           4       0.47      0.34      0.40       337
           5       0.40      0.45      0.42       549

    accuracy                           0.40      2230
   macro avg       0.41      0.40      0.40      2230
weighted avg       0.43      0.40      0.41      2230

--------------------------------------------------
Training Random Forest with CountVectorizer...
-----------------Random Forest with CountVectorizer------------------

Results for Random Forest:
Accuracy: 0.442152466367713
Classification Report:
              precision    recall  f1-score

: 

LEMMATISER ?

In [ ]:

X = dataiemocap['Phrase']
#---------------------C'EST ICI QU'ON LEMMATISE AVEC WORD NET ---------------------------
X = X.apply(lemmatiser)
#----------------------------------------------------------------------------------------

y = dataiemocap['Emotion']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# Vectoriseurs à tester
vectorizers = {
    'CountVectorizer': CountVectorizer(),
    'TfidfVectorizer': TfidfVectorizer()
}

for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())        

Training SVM with CountVectorizer...
-----------------SVM with CountVectorizer------------------

Results for SVM:
Accuracy: 0.44349775784753365
Classification Report:
              precision    recall  f1-score   support

           0       0.22      0.38      0.28       178
           1       0.54      0.52      0.53       315
           2       0.49      0.38      0.43       509
           3       0.50      0.46      0.48       342
           4       0.46      0.36      0.40       337
           5       0.44      0.52      0.48       549

    accuracy                           0.44      2230
   macro avg       0.44      0.44      0.43      2230
weighted avg       0.46      0.44      0.45      2230

--------------------------------------------------
Training Random Forest with CountVectorizer...
-----------------Random Forest with CountVectorizer------------------

Results for Random Forest:
Accuracy: 0.4798206278026906
Classification Report:
              precision    recall  f1-sco

MODIFIER TAILLE APPRENTISSAGE ET TAILLE TEST

0.3 -> 0,2

In [ ]:

X = dataiemocap['Phrase']
#---------------------C'EST ICI QU'ON LEMMATISE AVEC WORD NET ---------------------------
X = X.apply(lemmatiser)
#----------------------------------------------------------------------------------------

y = dataiemocap['Emotion']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Vectoriseurs à tester
vectorizers = {
    'CountVectorizer': CountVectorizer(),
    'TfidfVectorizer': TfidfVectorizer()
}

for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())        

Training SVM with CountVectorizer...
-----------------SVM with CountVectorizer------------------

Results for SVM:
Accuracy: 0.46402151983860124
Classification Report:
              precision    recall  f1-score   support

           0       0.21      0.39      0.27       114
           1       0.55      0.51      0.53       218
           2       0.52      0.40      0.45       341
           3       0.60      0.51      0.55       240
           4       0.56      0.42      0.48       227
           5       0.41      0.52      0.46       347

    accuracy                           0.46      1487
   macro avg       0.48      0.46      0.46      1487
weighted avg       0.50      0.46      0.47      1487

--------------------------------------------------
Training Random Forest with CountVectorizer...
-----------------Random Forest with CountVectorizer------------------

Results for Random Forest:
Accuracy: 0.5090786819098857
Classification Report:
              precision    recall  f1-sco

0.2 -> 0.4

In [ ]:

X = dataiemocap['Phrase']
#---------------------C'EST ICI QU'ON LEMMATISE AVEC WORD NET ---------------------------
X = X.apply(lemmatiser)
#----------------------------------------------------------------------------------------

y = dataiemocap['Emotion']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=2)

# Vectoriseurs à tester
vectorizers = {
    'CountVectorizer': CountVectorizer(),
    'TfidfVectorizer': TfidfVectorizer()
}

for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())        

Training SVM with CountVectorizer...
-----------------SVM with CountVectorizer------------------

Results for SVM:
Accuracy: 0.4320780094149294
Classification Report:
              precision    recall  f1-score   support

           0       0.23      0.41      0.29       254
           1       0.47      0.50      0.48       412
           2       0.46      0.36      0.40       689
           3       0.48      0.48      0.48       450
           4       0.52      0.34      0.41       428
           5       0.44      0.50      0.47       741

    accuracy                           0.43      2974
   macro avg       0.44      0.43      0.42      2974
weighted avg       0.45      0.43      0.43      2974

--------------------------------------------------
Training Random Forest with CountVectorizer...
-----------------Random Forest with CountVectorizer------------------

Results for Random Forest:
Accuracy: 0.4734364492266308
Classification Report:
              precision    recall  f1-scor

In [ ]:



#Movie datasets
dataiemocap = preprocess_comma("output.csv","dataiemocap.csv")
#REDUCTION DU DATAFRAME CAR TROP D'INSTANCES 






X = dataiemocap['Phrase']
#---------------------C'EST ICI QU'ON LEMMATISE AVEC WORD NET ---------------------------
X = X.apply(lemmatiser)
#----------------------------------------------------------------------------------------
y = dataiemocap['Emotion']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Vectoriseurs à tester
vectorizers = {
    'CountVectorizer': CountVectorizer(),
    'TfidfVectorizer': TfidfVectorizer()
}

for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
        
print(y_train.value_counts())
print(y_train_resampled.value_counts())      
  